In [1]:
import pandas as pd
from random import random
import sympy as sym
import numpy as np

pd.set_option('display.max_rows', None)

In [2]:
#Initial data-----------------------------------------------------------------------------------------------------

C10min=0     #min C10 value
C10max=10    #max C10 value
decimals=1   #number of decimal cases of C10 variable

#Training data
n=10               #number of C10 variables
st=3.0            #applied stretch
ninc=20           #increments number
dst=(st-1)/ninc   #Delta stretch

#Validation and Test Data
n_val=int(0.2*n)   # % of training data
n_test=int(0.1*n)  # % of training data
st=3.0             #applied stretch
#------------------------------------------------------------------------------------------------------------------

In [3]:
print(st/20)

0.15


In [4]:
C11 = sym.Symbol('C11')
C12 = sym.Symbol('C12')
C13 = sym.Symbol('C13')
C21 = sym.Symbol('C21')
C22 = sym.Symbol('C22')
C23 = sym.Symbol('C23')
C31 = sym.Symbol('C31')
C32 = sym.Symbol('C32')
C33 = sym.Symbol('C33')

In [5]:
C = sym.Matrix([[C11,C12,C13], [C21,C22,C23], [C31,C32,C33]]) 
I1=sym.trace(C)
Jac=sym.det(C)

#symbolic C10
C10s = sym.Symbol('C10s')

#Generate SEF
SEF=C10s*(I1-3)
#
#Second Piola Kirchoff Stresses
S11=2*sym.diff(SEF,C11)
S12=2*sym.diff(SEF,C12)
S13=2*sym.diff(SEF,C13)
S21=2*sym.diff(SEF,C21)
S22=2*sym.diff(SEF,C22)
S23=2*sym.diff(SEF,C23)
S31=2*sym.diff(SEF,C31)
S32=2*sym.diff(SEF,C32)
S33=2*sym.diff(SEF,C33)
#
S = sym.Matrix([[S11,S12,S13], [S21,S22,S23], [S31,S32,S33]])

In [6]:
#Training Data-----------------------------------------------------------------------------------------

#Generate random C10 and store them in a dataframe
C10df = pd.DataFrame()
for i in range(n):
    scaled_value = round(C10min + (random() * (C10max- C10min)),decimals)
    C10df.at[i,'C10']=scaled_value

In [7]:
#Calculate stresses
df_training = pd.DataFrame()
for i in range(len(C10df)):
    stj=1.0
    for j in range(int(ninc)):
        k=(ninc*i)+j
        stj=round((stj+dst),3)
        df_training.at[k,'stretch']=stj
        
        #Deformation Gradient
        F=sym.Matrix([[1/(np.sqrt(stj)),0,0], [0,stj,0], [0,0,1/(np.sqrt(stj))]]) 
        Ft=sym.transpose(F)
        C=Ft*F;

        C11v=C[0,0];
        C12v=C[0,1];
        C13v=C[0,2];
        C21v=C[1,0];
        C22v=C[1,1];
        C23v=C[1,2];
        C31v=C[2,0];
        C32v=C[2,1];
        C33v=C[2,2];

        Cauchy=(1/Jac)*(F*S*Ft)
    
        Stress22_Abaqus=Cauchy[1,1]-Cauchy[0,0]
    
        Cauchy_Abaqus=sym.Matrix([[0,0,0], [0, Stress22_Abaqus,0], [0,0,0]]) 
    
        df_training.at[k,'stress']=Stress22_Abaqus.subs([(C11, C11v), (C12, C12v), 
                                                         (C13, C13v),(C21, C21v), 
                                                         (C22, C22v), (C23, C23v),
                                                         (C31, C31v), (C32, C32v), 
                                                         (C33, C33v),(C10s,C10df.C10[i])])
        df_training.at[k,'C10']=C10df.C10[i]

print('-----Training data----')
print(df_training)
print('')

#Save training values in a .csv file
df_training.to_csv('df_training.csv', index=False, header=True)
#-------------------------------------------------------------------------------------------------------------------

-----Training data----
     stretch             stress  C10
0        1.1   2.22672727272728  3.7
1        1.2   4.48933333333333  3.7
2        1.3   6.81369230769231  3.7
3        1.4   9.21828571428571  3.7
4        1.5   11.7166666666667  3.7
5        1.6   14.3190000000000  3.7
6        1.7   17.0330588235294  3.7
7        1.8   19.8648888888889  3.7
8        1.9   22.8192631578947  3.7
9        2.0   25.9000000000000  3.7
10       2.1   29.1101904761905  3.7
11       2.2   32.4523636363636  3.7
12       2.3   35.9286086956522  3.7
13       2.4   39.5406666666667  3.7
14       2.5   43.2900000000000  3.7
15       2.6   47.1778461538462  3.7
16       2.7   51.2052592592593  3.7
17       2.8   55.3731428571429  3.7
18       2.9   59.6822758620690  3.7
19       3.0   64.1333333333333  3.7
20       1.1   1.32400000000000  2.2
21       1.2   2.66933333333333  2.2
22       1.3   4.05138461538462  2.2
23       1.4   5.48114285714286  2.2
24       1.5   6.96666666666666  2.2
25       1.6   

In [8]:
#Validation and Test Data--------------------------------------------------------------------------------------------------

In [9]:
#Generate random increments and store them in a dataframe
nincdf = pd.DataFrame()
for i in range(n_val+n_test):
    ninc_val = int(10 + (random() * (ninc-10)))
    nincdf.at[i,'ninc_val']=ninc_val

In [10]:
#Generate random C10 and store them in a dataframe
C10df_val = pd.DataFrame()
for i in range(n_val+n_test):
    scaled_value_val = round(C10min + (random() * (C10max- C10min)),decimals)
    C10df_val.at[i,'C10']=scaled_value_val

In [11]:
#Generate SEFs for each C10 and store them in a dataframe
SEFdf_val = pd.DataFrame()
for i in range(len(C10df_val)):
    C10_val=C10df_val.C10[i]
    SEFdf_val.at[i,'SEF']=C10_val*(I1-3)
#

In [12]:
#Calculate stresses
df_val = pd.DataFrame()
df_test = pd.DataFrame()
c=-1
for i in range(len(SEFdf_val)):
    dst=(st-1)/nincdf.ninc_val[i]
    stj=1.0
    c=c+1
    if i==n_val:
        c=0
    for j in range(int(nincdf.ninc_val[i])):   
        k=(nincdf.ninc_val[i]*c)+j
        stj=stj+dst
    
        #Deformation Gradient
        F=sym.Matrix([[1/(np.sqrt(stj)),0,0], [0,stj,0], [0,0,1/(np.sqrt(stj))]]) 
        Ft=sym.transpose(F)
        C=Ft*F;

        C11v=C[0,0];
        C12v=C[0,1];
        C13v=C[0,2];
        C21v=C[1,0];
        C22v=C[1,1];
        C23v=C[1,2];
        C31v=C[2,0];
        C32v=C[2,1];
        C33v=C[2,2];

        Cauchy=(1/Jac)*(F*S*Ft)
    
        Stress22_Abaqus=Cauchy[1,1]-Cauchy[0,0]
    
        Cauchy_Abaqus=sym.Matrix([[0,0,0], [0, Stress22_Abaqus,0], [0,0,0]]) 
        
        if i>(n_val-1):
            df_test.at[k,'stretch']=stj
            df_test.at[k,'stress']=Stress22_Abaqus.subs([(C11, C11v), (C12, C12v), 
                                                         (C13, C13v),(C21, C21v), 
                                                         (C22, C22v), (C23, C23v),
                                                         (C31, C31v), (C32, C32v), 
                                                         (C33, C33v),(C10s,C10df_val.C10[i])])
            df_test.at[k,'C10']=C10df_val.C10[i]
        else:
            df_val.at[k,'stretch']=stj
            df_val.at[k,'stress']=Stress22_Abaqus.subs([(C11, C11v), (C12, C12v), 
                                                        (C13, C13v),(C21, C21v), 
                                                        (C22, C22v), (C23, C23v),
                                                        (C31, C31v), (C32, C32v), 
                                                        (C33, C33v),(C10s,C10df_val.C10[i])])
            df_val.at[k,'C10']=C10df_val.C10[i]

print('-----Validation data----')
print(df_val)
print('')
print('-----Test data----')
print(df_test)

-----Validation data----
       stretch            stress  C10
0.0   1.166667  5.34206349206350  5.3
1.0   1.333333  10.8944444444445  5.3
2.0   1.500000  16.7833333333333  5.3
3.0   1.666667  23.0844444444445  5.3
4.0   1.833333  29.8459595959596  5.3
5.0   2.000000  37.1000000000000  5.3
6.0   2.166667  44.8688034188034  5.3
7.0   2.333333  53.1682539682540  5.3
8.0   2.500000  62.0100000000000  5.3
9.0   2.666667  71.4027777777778  5.3
10.0  2.833333  81.3532679738562  5.3
11.0  3.000000  91.8666666666666  5.3
18.0  1.111111  2.67654320987654  4.0
19.0  1.222222  5.40516273849608  4.0
20.0  1.333333  8.22222222222223  4.0
21.0  1.444444  11.1528964862298  4.0
22.0  1.555556  14.2151675485009  4.0
23.0  1.666667  17.4222222222222  4.0
24.0  1.777778  20.7839506172840  4.0
25.0  1.888889  24.3079157588962  4.0
26.0  2.000000  28.0000000000000  4.0
27.0  2.111111  31.8648473034438  4.0
28.0  2.222222  35.9061728395062  4.0
29.0  2.333333  40.1269841269841  4.0
30.0  2.444444  44.529741

In [13]:
#Save validation data in a .csv file
df_val.to_csv('df_val.csv', index=False, header=True)

#Save test data in a .csv file
df_val.to_csv('df_val.csv', index=False, header=True)